In [2]:
from SimulateTrading import *
import nest_asyncio
# from RealTrading import *
import TradeComputation
from pprint import pprint
import pandas as pd

if __name__ == "__main__":
    nest_asyncio.apply()
    pd.options.display.float_format = '{:,.4f}'.format

    symbols = [
        "btcusdt",
        "xrpusdt",
        "adausdt",
        "linkusdt",
        "sandusdt",
        "bnbusdt",
        "dogeusdt",
        "solusdt",
    ]  # 확인하고자 하는 심볼 정보
    # intervals = ["1m", "5m", "15m"]  # 백테스트 적용 interval값(다운로드 항목)
    ### 수신받을 데이터의 기간 ###
    start_date = "2024-11-1 09:00:00"  # 시작 시간
    end_date = "2024-11-12 08:59:59"  # 종료 시간
    safety_balance_ratio = 0.2  # 잔고 안전금액 지정 비율
    stop_loss_rate = 0.35  # 스톱 로스 비율
    is_download = True  # 기존 데이터로 할경우 False, 신규 다운로드 True
    adj_timer = True  # 시간 흐름에 따른 시작가 변동률 반영(stoploss에 영향미침.)
    adj_rate = 0.0007
    use_scale_stop = True  # final손절(False), Scale손절(True)
    seed_money = 69690
    max_trade_number = 3
    start_step = 5_000
    leverage = 15
    init_stop_rate = 0.015
    adj_interval = "3m"
    is_order_break = True
    loss_chance = 1
    step_interval = "4h"

    ### Ticker Setting Option ###
    comparison = "above"  # above : 이상, below : 이하
    absolute = True  # True : 비율 절대값, False : 비율 실제값
    value = 35_000_000  # 거래대금 : 단위 USD
    target_percent = 0.03  # 변동 비율폭 : 음수 가능
    quote_type = "usdt"  # 쌍거래 거래화폐

    backtest_ins = BackTester(
        symbols=symbols,
        # intervals=intervals,
        start_date=start_date,
        end_date=end_date,
        safety_balance_ratio=safety_balance_ratio,
        stop_loss_rate=stop_loss_rate,
        is_download=is_download,
        adj_timer=adj_timer,
        adj_rate=adj_rate,
        is_use_scale_stop=use_scale_stop,
        seed_money=seed_money,
        max_trade_number=max_trade_number,
        start_step=start_step,
        max_leverage=leverage,
        init_stop_rate=init_stop_rate,
        adj_interval=adj_interval,
        is_order_break=is_order_break,
        loss_chance=loss_chance,
        step_interval=step_interval,
        comparison=comparison,
        absolute=True,
        value=value,
        percent=target_percent,
        quote_type=quote_type,
    )

    asyncio.run(backtest_ins.run())
    analyze_statistics = TradeComputation.ResultEvaluator(backtest_ins.portfolio_ins)
    analyze_statistics.run_analysis()

BTCUSDT
1m
2024-11-01 09:00:00
2024-11-02 01:39:59
BTCUSDT
1m
2024-11-02 01:40:00
2024-11-02 18:19:59
BTCUSDT
1m
2024-11-02 18:20:00
2024-11-03 10:59:59
BTCUSDT
1m
2024-11-03 11:00:00
2024-11-04 03:39:59
BTCUSDT
1m
2024-11-04 03:40:00
2024-11-04 20:19:59
BTCUSDT
1m
2024-11-04 20:20:00
2024-11-05 12:59:59
BTCUSDT
1m
2024-11-05 13:00:00
2024-11-06 05:39:59
BTCUSDT
1m
2024-11-06 05:40:00
2024-11-06 22:19:59
BTCUSDT
1m
2024-11-06 22:20:00
2024-11-07 14:59:59
BTCUSDT
1m
2024-11-07 15:00:00
2024-11-08 07:39:59
BTCUSDT
1m
2024-11-08 07:40:00
2024-11-09 00:19:59
BTCUSDT
1m
2024-11-09 00:20:00
2024-11-09 16:59:59
BTCUSDT
1m
2024-11-09 17:00:00
2024-11-10 09:39:59
BTCUSDT
1m
2024-11-10 09:40:00
2024-11-11 02:19:59
BTCUSDT
1m
2024-11-11 02:20:00
2024-11-11 18:59:59
BTCUSDT
1m
2024-11-11 19:00:00
2024-11-12 08:59:59
BTCUSDT
5m
2024-11-01 09:00:00
2024-11-04 20:19:59
BTCUSDT
5m
2024-11-04 20:20:00
2024-11-08 07:39:59
BTCUSDT
5m
2024-11-08 07:40:00
2024-11-11 18:59:59
BTCUSDT
5m
2024-11-11 19:00:00


ValueError: 중간점검

In [3]:
history = backtest_ins.portfolio_ins.trade_history

In [4]:
backtest_ins.closing_sync_data

{'BTCUSDT': {'1m': array([[1.73041920e+12, 7.03219000e+04, 7.03454000e+04, ...,
          3.25386000e+02, 2.28810065e+07, 0.00000000e+00],
         [1.73041926e+12, 7.03432000e+04, 7.03528000e+04, ...,
          6.87670000e+01, 4.83715631e+06, 0.00000000e+00],
         [1.73041932e+12, 7.03457000e+04, 7.03584000e+04, ...,
          6.79580000e+01, 4.77989413e+06, 0.00000000e+00],
         ...,
         [1.73136942e+12, 8.85155000e+04, 8.86173000e+04, ...,
          1.15299000e+02, 1.02124544e+07, 0.00000000e+00],
         [1.73136948e+12, 8.86012000e+04, 8.87282000e+04, ...,
          1.81138000e+02, 1.60630775e+07, 0.00000000e+00],
         [1.73136954e+12, 8.87158000e+04, 8.87667000e+04, ...,
          1.41659000e+02, 1.25703567e+07, 0.00000000e+00]]),
  '5m': array([[1.73041920e+12, 7.03219000e+04, 7.03454000e+04, ...,
          3.25386000e+02, 2.28810065e+07, 0.00000000e+00],
         [1.73041920e+12, 7.03219000e+04, 7.03528000e+04, ...,
          3.94153000e+02, 2.77181629e+07, 0.

In [5]:
range_ = backtest_ins.closing_indices_data.get_data('interval_1h')[3100]

In [10]:
selec_data = backtest_ins.closing_sync_data['XRPUSDT']['1h'][range_-1]

In [15]:
selec_data

array([[1.73043000e+12, 5.10900000e-01, 5.19300000e-01, 5.10200000e-01,
        5.19300000e-01, 3.71912884e+07, 1.73043360e+12, 1.91544457e+07,
        3.92890000e+04, 1.96921823e+07, 1.01409950e+07, 0.00000000e+00],
       [1.73043360e+12, 5.19300000e-01, 5.21500000e-01, 5.15100000e-01,
        5.16400000e-01, 5.00107548e+07, 1.73043720e+12, 2.59120024e+07,
        4.61750000e+04, 2.28516448e+07, 1.18361903e+07, 0.00000000e+00],
       [1.73043720e+12, 5.16400000e-01, 5.16900000e-01, 5.13200000e-01,
        5.14700000e-01, 2.84338319e+07, 1.73044080e+12, 1.46382358e+07,
        2.61320000e+04, 1.40896061e+07, 7.25376633e+06, 0.00000000e+00],
       [1.73044080e+12, 5.14700000e-01, 5.16000000e-01, 5.12900000e-01,
        5.13100000e-01, 1.93336795e+07, 1.73044440e+12, 9.94612524e+06,
        2.38900000e+04, 9.23925220e+06, 4.75412672e+06, 0.00000000e+00],
       [1.73044440e+12, 5.13100000e-01, 5.14800000e-01, 5.09400000e-01,
        5.11900000e-01, 3.23702190e+07, 1.73044800e+12, 1.65

In [14]:
kline_array

NameError: name 'kline_array' is not defined

In [ ]:
k